# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [171]:
# import library
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from sqlalchemy import create_engine
%matplotlib inline

import pprint as pp

# import ml library
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,f1_score,recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# import pri library
import wrangling3 as w

# set env
## show all row and column
from IPython.display import display
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
## show 200 string in context
pd.set_option('max_colwidth',200)

In [172]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',con=engine)
df.head(1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that could pass over Haiti,Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [173]:
df.genre.value_counts()

news      13054
direct    10766
social     2396
Name: genre, dtype: int64

In [174]:
X = df['message']
Y = df[df.columns[5:]]
X.sample()

7150    we not find nothing, and i have an orphanage under my responsible with all people in the street MBC imp.des #2! 
Name: message, dtype: object

In [175]:
Y.sample()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
21052,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [176]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

In [177]:
tokenize(X[0])

['weather',
 'update',
 '-',
 'a',
 'cold',
 'front',
 'from',
 'cuba',
 'that',
 'could',
 'pas',
 'over',
 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [178]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [179]:
X = df.message
X.head(1)

0    Weather update - a cold front from Cuba that could pass over Haiti
Name: message, dtype: object

In [180]:
y = df[df.columns[5:]]
#Y = df.loc[:,df.columns[4:]]
y.head(1)

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [181]:
import wrangling3 as w
#w.checkvalue(y)

In [182]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [183]:
type(y_test)

pandas.core.frame.DataFrame

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [184]:
model = pipeline

In [185]:
model.fit(X_train, y_train);

C:\Users\mengf\Miniconda3\envs\py37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [206]:
y_pred = model.predict(X_test)

In [202]:
cols = df.columns[5:]

In [207]:
y_pred = pd.DataFrame(y_pred,columns=cols)

In [222]:
y_pred.sample()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
5357,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [223]:
for i in cols:
    print('- - - {} - - -'.format(i))
    print(classification_report(y_test[i],y_pred[i], digits=4))

- - - request - - -
              precision    recall  f1-score   support

           0     0.8855    0.9829    0.9317      5424
           1     0.8258    0.3903    0.5300      1130

    accuracy                         0.8807      6554
   macro avg     0.8557    0.6866    0.7309      6554
weighted avg     0.8753    0.8807    0.8624      6554

- - - offer - - -


C:\Users\mengf\Miniconda3\envs\py37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0     0.9959    1.0000    0.9979      6527
           1     0.0000    0.0000    0.0000        27

    accuracy                         0.9959      6554
   macro avg     0.4979    0.5000    0.4990      6554
weighted avg     0.9918    0.9959    0.9938      6554

- - - aid_related - - -
              precision    recall  f1-score   support

           0     0.6917    0.9002    0.7823      3868
           1     0.7461    0.4222    0.5392      2686

    accuracy                         0.7043      6554
   macro avg     0.7189    0.6612    0.6608      6554
weighted avg     0.7140    0.7043    0.6827      6554

- - - medical_help - - -
              precision    recall  f1-score   support

           0     0.9247    0.9988    0.9604      6052
           1     0.5882    0.0199    0.0385       502

    accuracy                         0.9239      6554
   macro avg     0.7565    0.5094    0.4994      6554
weighted avg     0.8990  

              precision    recall  f1-score   support

           0     0.9890    1.0000    0.9945      6482
           1     0.0000    0.0000    0.0000        72

    accuracy                         0.9890      6554
   macro avg     0.4945    0.5000    0.4972      6554
weighted avg     0.9781    0.9890    0.9836      6554

- - - other_infrastructure - - -
              precision    recall  f1-score   support

           0     0.9544    0.9994    0.9763      6255
           1     0.0000    0.0000    0.0000       299

    accuracy                         0.9538      6554
   macro avg     0.4772    0.4997    0.4882      6554
weighted avg     0.9108    0.9538    0.9318      6554

- - - weather_related - - -
              precision    recall  f1-score   support

           0     0.8210    0.9751    0.8914      4774
           1     0.8654    0.4298    0.5743      1780

    accuracy                         0.8270      6554
   macro avg     0.8432    0.7024    0.7329      6554
weighted avg 

In [219]:
# archive
# 如果不转换 y_pred 需要进行转换后比较
# print(classification_report(y_test.iloc[:,0], y_pred.T[0]))；

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.